In [1]:
using Interact
#using PyPlot #no se ocupa
using LinearAlgebra
using PyCall
using Plots
pygui(:qt) 

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-13911393718295420582\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9064308556907921447\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/45c644a0e207fcf61ef0bbb18ceccb0f56f40558-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-13911393718295420582\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

:qt

In [32]:
P₁ = [[-5.58,-1.43],
[-6.49,-1.38],
[-6.2,-0.6],
[-6.16,0],
[-1,4],
[-0.18,4.66],
[0.6,4.43],
[1.16,4.55],
[1.77,1.29],
[0.89,-0.17],
[0.21,-0.95],
[1.16,0.9]] #Datos del triángulo

P₁ = hcat(P₁...)'

P₂ = [[3.33,-4],
[2.49,-3.94],
[2.51,-3.34],
[1.94,-2.64],
[3.49,1.39],
[3.65,2.31],
[4.72,2.7],
[5.12,3.44],
[8.54,2.46],
[9.39,2.48],
[9.73,1.63],
[10.68,0.85],
[9.27,-1.77],
[8.93,-2.35],
[8.5,-2.76],
[7.4,-2.98]] #Datos del cuadrilatero

P₂ = hcat(P₂...)'

16×2 Adjoint{Float64,Array{Float64,2}}:
  3.33  -4.0 
  2.49  -3.94
  2.51  -3.34
  1.94  -2.64
  3.49   1.39
  3.65   2.31
  4.72   2.7 
  5.12   3.44
  8.54   2.46
  9.39   2.48
  9.73   1.63
 10.68   0.85
  9.27  -1.77
  8.93  -2.35
  8.5   -2.76
  7.4   -2.98

In [19]:
A₂ = [1, -5]
B₂ = [4, 4]
C₂ = [12, 2]
D₂ = [9, -3]

2-element Array{Int64,1}:
  9
 -3

In [20]:
B₁ = [1, 6]
A₁ = [-9, -2]
C₁ = [3, -1]

2-element Array{Int64,1}:
  3
 -1

## Nota: los siguientes rutinas son para Ovalos.jl -----------------------------------------

In [4]:
function curve(P, Q, R, ω, t)
    p(x) = (1-x)^2; 
    q(x) = 2*x*(1-x); 
    s(x) = x^2; 
    return (P*p(t)+Q*ω*q(t)+s(t)*R)/(p(t)+q(t)*ω+s(t))
end
    
function linRange_curve(P, Q, R, ω, t)
    c = map( x -> curve(P, Q, R, ω, x), t)
    return hcat(c...)'
end

linRange_curve (generic function with 1 method)

In [21]:
function plot_data(p)
    x = p[1]
    y = p[2]
    Plots.plot!(x, y, legend=false, size=(600,500), seriestype=:scatter)
end

plot_data (generic function with 1 method)

In [38]:
function ovalo3(A, B, C, ω, t, P)  
    p_ab = (1-t[1])*A + t[1]*B
    p_bc = (1-t[2])*B + t[2]*C
    p_ca = (1-t[3])*C + t[3]*A
    Tr = [A'; B'; C'; A']
    Tr2 = [p_ab'; p_bc'; p_ca'; p_ab']
    T = LinRange(0, 1, 100) 
    Cₐ = linRange_curve(p_ab, A, p_ca, ω[1], T)
    Cᵦ = linRange_curve(p_bc, B, p_ab, ω[2], T)
    Cᵧ = linRange_curve(p_bc, C, p_ca, ω[3], T)
    plt = Plots.plot(Tr[:,1], Tr[:,2], color="blue", legend=false, size=(600,500))
    Plots.plot!(Tr2[:,1], Tr2[:,2], color="red", legend=false, size=(600,500), seriestype=:scatter)
    Plots.plot!(Cₐ[:,1], Cₐ[:,2], color="black", legend=false, size=(600, 500))
    Plots.plot!(Cᵦ[:,1], Cᵦ[:,2], color="black", legend=false, size=(600, 500))
    Plots.plot!(Cᵧ[:,1], Cᵧ[:,2], color="black", legend=false, size=(600, 500))
    Plots.plot!(P[:,1], P[:,2], color="orange", legend=false, size=(600, 500), seriestype=:scatter)
end

ovalo3 (generic function with 2 methods)

In [43]:
function ovalo4(A, B, C, D, ω, t, P)
    p_ab = (1-t[1])*A + t[1]*B
    p_bc = (1-t[2])*B + t[2]*C
    p_cd = (1-t[3])*C + t[3]*D
    p_da = (1-t[4])*D + t[4]*A
    Tr = [A'; B'; C'; D'; A']
    Tr2 = [p_ab'; p_bc'; p_cd'; p_da'; p_ab']
    T = LinRange(0, 1, 100) 
    Cₐ = linRange_curve(p_ab, A, p_da, ω[1], T)
    Cᵦ = linRange_curve(p_bc, B, p_ab, ω[2], T)
    Cᵧ = linRange_curve(p_bc, C, p_cd, ω[3], T)
    Cᵪ = linRange_curve(p_cd, D, p_da, ω[4], T)
    plt = Plots.plot(Tr[:,1], Tr[:,2], color="blue", legend=false, size=(600,500))
    Plots.plot!(Tr2[:,1], Tr2[:,2], color="red", legend=false, size=(600,500), seriestype=:scatter)
    Plots.plot!(Cₐ[:,1], Cₐ[:,2], color="black", legend=false, size=(600, 500))
    Plots.plot!(Cᵦ[:,1], Cᵦ[:,2], color="black", legend=false, size=(600, 500))
    Plots.plot!(Cᵧ[:,1], Cᵧ[:,2], color="black", legend=false, size=(600, 500))
    Plots.plot!(Cᵪ[:,1], Cᵪ[:,2], color="black", legend=false, size=(600, 500))
    Plots.plot!(P[:,1], P[:,2], color="orange", legend=false, size=(600, 500), seriestype=:scatter)
end

ovalo4 (generic function with 2 methods)

## Aquí acaba Ovalos.jl -----------------------------------------

In [40]:
function interaccion(bool, A, B, C, D, P)
    
    if bool == false
        @manipulate for t₁ = 0:0.01:1, t₂ = 0:0.01:1, t₃ = 0:0.01:1, 
            ω₁ = 0:0.01:5, ω₂ = 0:0.01:5, ω₃ = 0:0.01:5
            t = [t₁, t₂, t₃]; ω = [ω₁, ω₂, ω₃]
            ovalo3(A, B, C, ω, t, P)
        end
    else
        @manipulate for t₁ = 0:0.01:1, t₂ = 0:0.01:1, t₃ = 0:0.01:1, t₄ = 0:0.01:1,
            ω₁ = 0:0.01:5, ω₂ = 0:0.01:5, ω₃ = 0:0.01:5, ω₄ = 0:0.01:5
            t = [t₁, t₂, t₃, t₄]; ω = [ω₁, ω₂, ω₃, ω₄]
            ovalo4(A, B, C, D, ω, t, P)
        end
    end
    
end

interaccion (generic function with 2 methods)

In [44]:
interaccion(false, A₁, B₁, C₁, D₂, P₁)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-fc8d2ca1-483e-4686-b348-b9dc5a4c5d27", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["t₁"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>101,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Any} with 2 listeners. Value:
51, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/sergio/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/sergio/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/sergio/.julia/packages/InteractBase/R8JgH/src/../assets/all.js"), Asset("css", nothing, "/Users/sergio/.julia/packages/InteractBase/R8JgH/src/../assets/style.css"), Asset("css", nothing, "/Users/sergio/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\",\"0.21\",\"0.22\",\"0.23\",\"0.24\",\"0.25\",\"0.26\",\"0.27\",\"0.28\",\"0.29\",\"0.3\",\"0.31\",\"0.32\",\"0.33\",\"0.34\",\"0.35\",\"0.36\",\"0.37\",\"0.38\",\"0.39\",\"0.4\",\"0.41\",\"0.42\",\"0.43\",\"0.44\",\"0.45\",\"0.46\",\"0.47\",\"0.48\",\"0.49\",\"0.5\",\"0.51\",\"0.52\",\"0.53\",\"0.54\",\"0.55\",\"0.56\",\"0.57\",\"0.58\",\"0.59\",\"0.6\",\"0.61\",\"0.62\",\"

In [45]:
interaccion(true, A₂, B₂, C₂, D₂, P₂)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-ca244987-7e6c-4023-902f-3202d1301c5d", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["t₁"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>101,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Any} with 2 listeners. Value:
51, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/sergio/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/sergio/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/sergio/.julia/packages/InteractBase/R8JgH/src/../assets/all.js"), Asset("css", nothing, "/Users/sergio/.julia/packages/InteractBase/R8JgH/src/../assets/style.css"), Asset("css", nothing, "/Users/sergio/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\",\"0.21\",\"0.22\",\"0.23\",\"0.24\",\"0.25\",\"0.26\",\"0.27\",\"0.28\",\"0.29\",\"0.3\",\"0.31\",\"0.32\",\"0.33\",\"0.34\",\"0.35\",\"0.36\",\"0.37\",\"0.38\",\"0.39\",\"0.4\",\"0.41\",\"0.42\",\"0.43\",\"0.44\",\"0.45\",\"0.46\",\"0.47\",\"0.48\",\"0.49\",\"0.5\",\"0.51\",\"0.52\",\"0.53\",\"0.54\",\"0.55\",\"0.56\",\"0.57\",\"0.58\",\"0.59\",\"0.6\",\"0.61\",\"0.62\",\"

In [2]:
#function interaccion3(A, B, C, P)
    # rutina de Modulo
 #   interaccion(false)
#end  

interaccion3 (generic function with 1 method)

In [3]:
#function interaccion4(A, B, C, D, P)
    # rutina de Modulo
#    interaccion(true)
#end    

interaccion4 (generic function with 1 method)